In [2]:
"""
− используйте данные MovieLens 1M
− можно использовать любые модели из пакета
− получите RMSE на тестовом сете 0.87 и ниже
"""

import pandas as pd
import numpy as np
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
# !pip install surprise

In [9]:
ratings = pd.read_csv('ratings.csv')
ratings.sample(25)

,userId,movieId,rating,timestamp
74336,474,4769,3.0,1056372207
84724,547,2076,5.0,942723440
2115,18,59615,2.5,1455231357
94769,599,79702,2.5,1498517021
76040,478,33164,0.5,1121467489
28240,195,3826,3.0,997909336
48352,313,1374,4.0,1030474742
62274,413,4890,5.0,1484440124
33855,230,33679,4.5,1196305197
30296,212,1265,2.5,1490121496


In [6]:
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [26]:
# В переменную movies_ratings положим 2 таблицы - с рейтингами и фильмами
# (для метода join сначала создадим, а затем удалим индекс по полю movieId, заодно удалив пропуски и переписав датасет)
movies_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)
movies_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [12]:
# Выберем наугад пользователя с его рейтингами 
movies_ratings[movies_ratings.userId == 230].title.unique()

array(['Jumanji (1995)', 'Babe (1995)', 'Clueless (1995)',
       'Seven (a.k.a. Se7en) (1995)', 'Apollo 13 (1995)',
       'Batman Forever (1995)', 'Hackers (1995)',
       'Dumb & Dumber (Dumb and Dumber) (1994)',
       'Interview with the Vampire: The Vampire Chronicles (1994)',
       'Miracle on 34th Street (1994)',
       'Shawshank Redemption, The (1994)',
       'While You Were Sleeping (1995)',
       'Ace Ventura: Pet Detective (1994)', 'Lion King, The (1994)',
       'Mask, The (1994)', 'Speed (1994)', 'True Lies (1994)',
       "Schindler's List (1993)", 'Sleepless in Seattle (1993)',
       'Home Alone (1990)', 'Ghost (1990)', 'Aladdin (1992)',
       'Terminator 2: Judgment Day (1991)', 'Dances with Wolves (1990)',
       'Batman (1989)', 'Silence of the Lambs, The (1991)',
       'Beauty and the Beast (1991)', 'Pretty Woman (1990)',
       'Aristocats, The (1970)', 'Mission: Impossible (1996)',
       'James and the Giant Peach (1996)', 'Twister (1996)',
       'Indepen

In [21]:
# Создадим датафрейм Pandas 
# dataset = pd.DataFrame({
#     'uid': movies_ratings.userId,       # id пользователя
#     'iid': movies_ratings.title,        # название фильма
#     'rating': movies_ratings.rating     # рейтинг, который пользователь поставил фильму 
# })
# dataset[dataset.uid == 230].iid.unique()

In [24]:
# Изучим разброс значений по рейтингам в датафрейме 
print(ratings.rating.min())
print(ratings.rating.max())

0.5
5.0


In [28]:
# Объявим переменную класса Reader и преобразуем датафрейм Pandas в датасет surprise 
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(movies_ratings[['userId', 'title', 'rating']], reader)

In [29]:
# Разобьем датасет surprise на трейн и тест 
trainset, testset = train_test_split(data, test_size=0.10)

In [30]:
algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [31]:
# Сформируем прогноз на основе полученной модели
test_pred = algo.test(testset)

In [33]:
# Проверим качество прогноза
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9608


0.9607602347017876

In [41]:
# Попробуем повысить качество прогноза (уменьшим ошибку). 
# Испытаем гипотезу, снизив количество похожих пользователей до 25-ти
algo = KNNBasic(k=25, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [37]:
# Сформируем новый прогноз на основе полученной модели и проверим качество 
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9619


0.9618514027340799

In [42]:
# Применим алгоритм SVD (кросс-валидация): требуемая точность достигнута 
from surprise import SVD
from surprise.model_selection import cross_validate

algo_svd = SVD()
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8663  0.8799  0.8767  0.8720  0.8745  0.8739  0.0046  
MAE (testset)     0.6656  0.6732  0.6755  0.6718  0.6714  0.6715  0.0033  
Fit time          5.34    5.05    5.08    5.22    5.19    5.17    0.11    
Test time         0.22    0.21    0.14    0.17    0.14    0.18    0.03    


{'test_rmse': array([0.8663193 , 0.87987362, 0.87665663, 0.87196777, 0.87454655]),
 'test_mae': array([0.66560173, 0.67322859, 0.67554245, 0.67181064, 0.6713536 ]),
 'fit_time': (5.340404748916626,
  5.04597544670105,
  5.078656435012817,
  5.21901535987854,
  5.189015626907349),
 'test_time': (0.22144651412963867,
  0.20526623725891113,
  0.1399376392364502,
  0.1683351993560791,
  0.14021730422973633)}

In [44]:
# Попробуем также использовать модель KNNWithMeans
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8833


0.8832553077790254

In [46]:
# Попробуем также использовать модель KNNWithMeans
algo = KNNWithMeans(k=25, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8827


0.8826541742067375

In [48]:
# Применим итератор перекретной проверки, который может быть использован и в других сетах
# Первое разбиение дает наилучший результат
from surprise.model_selection import KFold

kf = KFold(n_splits=4)
for trainset, testset in kf.split(data):

    # алгоритм обучения и прогнозирования 
    algo_svd.fit(trainset)
    predictions = algo_svd.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8722
RMSE: 0.8796
RMSE: 0.8746
RMSE: 0.8746


In [49]:
# Попробуем настроить параметры алгоритма с помощью метода GridSearchCV

from surprise.model_selection import GridSearchCV

# определяем набор параметров для тестирования 
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.007],
              'reg_all': [0.4, 0.6, 0.8]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=4)

gs.fit(data)

# выведем наилучший результат
print(gs.best_score['rmse'])

# комбинация параметров для получения наименьшей ошибки 
print(gs.best_params['rmse'])

0.8836173224315709
{'n_epochs': 15, 'lr_all': 0.007, 'reg_all': 0.4}
